# Building the Classifier

In [16]:
import pickle
import pandas as pd
import numpy as np
from os.path import join

from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels

## 1. For a single region

**1.1** Read saved feature matrix and corresponding labels

In [5]:
region = 'mixco_3'

pickle_path = join('..', '..', 'pickles')
with open(join(pickle_path, 'resnet50_features_' + region + '_train.pkl'), 'rb') as f:
    df_train = pickle.load(f)

**1.2** Convert the features into a matrix and the labels into a list

In [ ]:
feature_matrix = df_train['features'].to_numpy()
feature_matrix = np.column_stack(feat_matrix).transpose()

labels = df_train['label'].to_numpy().astype('int')

1.3 Split data into train and validation set

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(feature_matrix,
                                                                            labels,
                                                                            test_size=0.33,
                                                                            random_state=43)

1.3. Train a classifier on the training set and save it to disk

In [ ]:
clf = svm.SVC(gamma='scale', probability=True, C=100, decision_function_shape='ovr')
clf.fit(features_train, labels_train)

with open(join(pickle_path, 'classifier.pkl' ), 'wb') as f:
    pickle.dump(clf, f)

1.4. Predict labels on the validation set according to the classifier

In [ ]:
predicted_labels = clf.predict(features_test)

1.5. Calculate and plot (to do) the confusion matrix

In [ ]:
cm = confusion_matrix(labels_test, predicted_labels)
print(cm)

In [ ]:
pred_probas = clf.predict_proba(features_test)

for i in range(len(predicted_labels)):
    print(pred_probas[i], labels_test[i])

## 2. Build Classifier Using All Regions

2.1 Load all available training features into one big dataframe

In [30]:
regions = ['mixco_3'] # add the other regions!

In [25]:
columns = ['id', 'features', 'label']
df_train_global = pd.DataFrame(columns=columns)

for region in regions:
    try:
        with open(join(pickle_path, 'resnet50_features_' + region + '_train.pkl'), 'rb') as f:
            df_train = pickle.load(f)
            df_train_global = pd.concat([df_train_global, df_train])
    except:
        print("Error reading training data for region ", region)
    

In [26]:
df_train_global.head()

,id,features,label
0,7a1ea402,"[1.060129, 7.0963063, 3.184064, 0.026273847, 0...",0
1,7a1ec590,"[3.051854, 2.693229, 1.2304544, 0.0, 0.0, 0.0,...",0
2,7a1f387c,"[3.9077654, 6.1235085, 3.0621212, 0.093847096,...",0
3,7a1f5d2a,"[3.250351, 5.1703796, 0.5719796, 1.2888488, 1....",0
4,7a1fb914,"[2.6564455, 3.5307846, 1.1352928, 0.20922577, ...",0


2.2 Expand the dataframe such that instead of a list of features, every feature has its one column

In [27]:
#features_train = df_train_global.features.apply(lambda features: pd.Series(list(features)))
#features_train

#features_train = df_train_global.features.apply(lambda x: pd.Series(list(x)))
#features_train = features_train.astype('double')

feature_matrix_global = df_train_global['features'].to_numpy()
feature_matrix_global = np.column_stack(feature_matrix_global).transpose()

labels_global = df_train_global['label'].to_numpy().astype('int')

In [28]:
feature_matrix_global.shape

(159, 2048)

**2.3** Train the classifier using all training data  

In [29]:
clf = svm.SVC(gamma='scale', probability=True, C=100, decision_function_shape='ovr')
clf.fit(feature_matrix_global, labels_global)

with open(join(pickle_path, 'classifier_global.pkl' ), 'wb') as f:
    pickle.dump(clf, f)